# In this notebook we will try to solve the prolem of overfitting by using L1 and L2 regularization technique.

In [683]:
import pandas as pd
import numpy as np


In [684]:
df = pd.read_csv('Melbourne_housing.csv')
df

/var/folders/r8/rcrrys2d27z0jbgj24qp9rww0000gn/T/ipykernel_65324/1321220460.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Melbourne_housing.csv')


,Suburb,Address,Rooms,Type,Method,SellerG,Date,Distance,Postcode,Bedroom,...,Landsize,BuildingArea,YearBuilt,CouncilArea,Latitude,Longtitude,Regionname,Propertycount,ParkingArea,Price
0,Abbotsford,68 Studley St,2,h,SS,Jellis,3/9/2016,2.5,3067.0,2.0,...,126.0,inf,NaN,Yarra City Council,-37.80140,144.99580,Northern Metropolitan,4019.0,Carport,NaN
1,Airport West,154 Halsey Rd,3,t,PI,Nelson,3/9/2016,13.5,3042.0,3.0,...,303.0,225,2016.0,Moonee Valley City Council,-37.71800,144.87800,Western Metropolitan,3464.0,Detached Garage,840000.0
2,Albert Park,105 Kerferd Rd,2,h,S,hockingstuart,3/9/2016,3.3,3206.0,2.0,...,120.0,82,1900.0,Port Phillip City Council,-37.84590,144.95550,Southern Metropolitan,3280.0,Attached Garage,1275000.0
3,Albert Park,85 Richardson St,2,h,S,Thomson,3/9/2016,3.3,3206.0,2.0,...,159.0,inf,NaN,Port Phillip City Council,-37.84500,144.95380,Southern Metropolitan,3280.0,Indoor,1455000.0
4,Alphington,30 Austin St,3,h,SN,McGrath,3/9/2016,6.4,3078.0,3.0,...,174.0,122,2003.0,Darebin City Council,-37.78180,145.01980,Northern Metropolitan,2211.0,Parkade,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34852,Reservoir,18 Elinda Pl,3,u,SP,RW,30/09/2017,12.0,3073.0,3.0,...,NaN,105.0,1990.0,Darebin City Council,-37.69769,145.02332,Northern Metropolitan,21650.0,Parkade,475000.0
34853,Roxburgh Park,14 Stainsby Cr,4,h,S,Raine,30/09/2017,20.6,3064.0,4.0,...,NaN,225.0,1995.0,Hume City Council,-37.63665,144.92976,Northern Metropolitan,5833.0,Underground,591000.0
34854,Springvale South,8 Bellbird Ct,4,h,PI,Barry,30/09/2017,22.2,3172.0,4.0,...,534.0,152.0,1970.0,Greater Dandenong City Council,-37.97037,145.15449,South-Eastern Metropolitan,4054.0,Carport,NaN
34855,Springvale South,30 Waddington Cr,3,h,S,Harcourts,30/09/2017,22.2,3172.0,3.0,...,544.0,NaN,NaN,Greater Dandenong City Council,-37.97751,145.14813,South-Eastern Metropolitan,4054.0,Detached Garage,780500.0


# Numero di valori nulli per ogni colonna

In [685]:
print(df.isnull().sum())

Suburb               0
Address              0
Rooms                0
Type                 0
Method               0
SellerG              0
Date                 0
Distance             1
Postcode             1
Bedroom           8217
Bathroom          8226
Car               8728
Landsize         11810
BuildingArea     21097
YearBuilt        19306
CouncilArea          3
Latitude          7976
Longtitude        7976
Regionname           0
Propertycount        3
ParkingArea          0
Price             7610
dtype: int64


# PULIZIA

In [686]:
# Elimino le colonne con troppi valori nulli
df = df.drop(columns=["BuildingArea", "YearBuilt"])

# Calcolo la media delle colonne numeriche
numeric_cols = df.select_dtypes(include=["float64", "int64"]).columns

# Riempio i NaN delle colonne numeriche con la media
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

print(df.isnull().sum())

Suburb           0
Address          0
Rooms            0
Type             0
Method           0
SellerG          0
Date             0
Distance         0
Postcode         0
Bedroom          0
Bathroom         0
Car              0
Landsize         0
CouncilArea      3
Latitude         0
Longtitude       0
Regionname       0
Propertycount    0
ParkingArea      0
Price            0
dtype: int64


Le colonne non numeriche (come CouncilArea, ParkingArea, ecc.) non vengono toccate -> mantengono eventuali NaN

In [687]:
# Sostituisco stringhe vuote "" con NaN (altrimenti dropna non le vede)
df = df.replace("", np.nan)

# Elimino le righe dove qualsiasi colonna stringa è vuota
string_cols = df.select_dtypes(include=["object"]).columns

df = df.dropna(subset=string_cols, how="any")

print(df.isnull().sum())

Suburb           0
Address          0
Rooms            0
Type             0
Method           0
SellerG          0
Date             0
Distance         0
Postcode         0
Bedroom          0
Bathroom         0
Car              0
Landsize         0
CouncilArea      0
Latitude         0
Longtitude       0
Regionname       0
Propertycount    0
ParkingArea      0
Price            0
dtype: int64


In [688]:
df.nunique()

Suburb             350
Address          34006
Rooms               12
Type                 3
Method               9
SellerG            388
Date                78
Distance           215
Postcode           211
Bedroom             16
Bathroom            12
Car                 16
Landsize          1685
CouncilArea         33
Latitude         13403
Longtitude       14525
Regionname           8
Propertycount      342
ParkingArea          8
Price             2872
dtype: int64

In [689]:
df.shape

(34854, 20)

## Colonne che usiamo

In [690]:
# now lets observe our data and try to drop some columns that are not very uselful in our data analysis. I am just dropping some columns
# like date, latitute etc that are not very meaningful for our analysis. 
columns_to_use = ['Suburb', 'Rooms', 'Type', 'Method', 'SellerG', 'Regionname', 'Propertycount', 'Distance', 'CouncilArea', 'Bedroom', 'Bathroom', 'Car', 'Landsize', 'Price', 'ParkingArea']
df_new = df[columns_to_use]
df_new

,Suburb,Rooms,Type,Method,SellerG,Regionname,Propertycount,Distance,CouncilArea,Bedroom,Bathroom,Car,Landsize,Price,ParkingArea
0,Abbotsford,2,h,SS,Jellis,Northern Metropolitan,4019.0,2.5,Yarra City Council,2.0,1.0,1.0,126.000000,1.050173e+06,Carport
1,Airport West,3,t,PI,Nelson,Western Metropolitan,3464.0,13.5,Moonee Valley City Council,3.0,2.0,1.0,303.000000,8.400000e+05,Detached Garage
2,Albert Park,2,h,S,hockingstuart,Southern Metropolitan,3280.0,3.3,Port Phillip City Council,2.0,1.0,0.0,120.000000,1.275000e+06,Attached Garage
3,Albert Park,2,h,S,Thomson,Southern Metropolitan,3280.0,3.3,Port Phillip City Council,2.0,1.0,0.0,159.000000,1.455000e+06,Indoor
4,Alphington,3,h,SN,McGrath,Northern Metropolitan,2211.0,6.4,Darebin City Council,3.0,2.0,1.0,174.000000,1.050173e+06,Parkade
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34852,Reservoir,3,u,SP,RW,Northern Metropolitan,21650.0,12.0,Darebin City Council,3.0,1.0,1.0,593.598993,4.750000e+05,Parkade
34853,Roxburgh Park,4,h,S,Raine,Northern Metropolitan,5833.0,20.6,Hume City Council,4.0,2.0,2.0,593.598993,5.910000e+05,Underground
34854,Springvale South,4,h,PI,Barry,South-Eastern Metropolitan,4054.0,22.2,Greater Dandenong City Council,4.0,2.0,2.0,534.000000,1.050173e+06,Carport
34855,Springvale South,3,h,S,Harcourts,South-Eastern Metropolitan,4054.0,22.2,Greater Dandenong City Council,3.0,2.0,1.0,544.000000,7.805000e+05,Detached Garage


In [691]:
df_new.shape

(34854, 15)

## Variabili dummy

In [692]:
# now we are good to go with out cleaned data. Now we are going to make dummy variables for our whole dataset.
df_new = pd.get_dummies(df_new, drop_first=True) # it is a short cut to avoid dummy variable trap it is just dropping the main column whose dummies we have produced. 
df_new

,Rooms,Propertycount,Distance,Bedroom,Bathroom,Car,Landsize,Price,Suburb_Aberfeldie,Suburb_Airport West,...,CouncilArea_Wyndham City Council,CouncilArea_Yarra City Council,CouncilArea_Yarra Ranges Shire Council,ParkingArea_Carport,ParkingArea_Detached Garage,ParkingArea_Indoor,ParkingArea_Outdoor Stall,ParkingArea_Parkade,ParkingArea_Parking Pad,ParkingArea_Underground
0,2,4019.0,2.5,2.0,1.0,1.0,126.000000,1.050173e+06,False,False,...,False,True,False,True,False,False,False,False,False,False
1,3,3464.0,13.5,3.0,2.0,1.0,303.000000,8.400000e+05,False,True,...,False,False,False,False,True,False,False,False,False,False
2,2,3280.0,3.3,2.0,1.0,0.0,120.000000,1.275000e+06,False,False,...,False,False,False,False,False,False,False,False,False,False
3,2,3280.0,3.3,2.0,1.0,0.0,159.000000,1.455000e+06,False,False,...,False,False,False,False,False,True,False,False,False,False
4,3,2211.0,6.4,3.0,2.0,1.0,174.000000,1.050173e+06,False,False,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34852,3,21650.0,12.0,3.0,1.0,1.0,593.598993,4.750000e+05,False,False,...,False,False,False,False,False,False,False,True,False,False
34853,4,5833.0,20.6,4.0,2.0,2.0,593.598993,5.910000e+05,False,False,...,False,False,False,False,False,False,False,False,False,True
34854,4,4054.0,22.2,4.0,2.0,2.0,534.000000,1.050173e+06,False,False,...,False,False,False,True,False,False,False,False,False,False
34855,3,4054.0,22.2,3.0,2.0,1.0,544.000000,7.805000e+05,False,False,...,False,False,False,False,True,False,False,False,False,False


# MACHINE LEARNING MODEL

In [693]:
X = df_new.drop('Price', axis='columns')
y = df_new.Price

## Split

In [694]:
# Now we can jump into our machine learning model and lets first use the train_test_split method
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=2)

## LinearRegression

In [695]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)


,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [696]:
from sklearn.metrics import mean_squared_error
y_pred_lin = model.predict(X_test)

rmse_lin = np.sqrt(mean_squared_error(y_test, y_pred_lin))
r2_lin = model.score(X_test, y_test)
print(f"TEST - [Linear Regr.] RMSE: {rmse_lin:.4f} | R2: {r2_lin:.4f}")
# Our model is much overfit with the training dataset that its accuracy in negative when we provide it with testing dataset. 

TEST - [Linear Regr.] RMSE: 397540.7133 | R2: 0.4819


In [697]:
from sklearn.metrics import mean_squared_error
y_pred_lin = model.predict(X_train)

mse_lin = np.sqrt(mean_squared_error(y_train, y_pred_lin))
r2_lin = model.score(X_train, y_train) # at the same our model is performing very well with respect to the training datset
print(f"TRAIN - [Linear Regr.] RMSE: {rmse_lin:.4f} | R2: {r2_lin:.4f}")

TRAIN - [Linear Regr.] RMSE: 397540.7133 | R2: 0.4928


## Lasso

In [698]:
# so we can see that our model is facing the problem of overfitting because on training dataset it scores higher and on the
# testing dataset it score lower. In simple words our model is overfit to the training dataset and underfit to the testing dataset.
# We can solve the problem of overfitting by using L1 0r L2 regularization.  
from sklearn.linear_model import Lasso    # Sklearn's Lass regression is the L1 regularization. 
lasso_model = Lasso()
lasso_model.fit(X_train, y_train)
# the L1 regularization or the Lasso model will add an absolute θ value in the mean squared error

/Users/ashura/Documents/GitHub/Anzellotti_DepositoCorso/.venv/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.306e+15, tolerance: 9.084e+11
  model = cd_fast.enet_coordinate_descent(


,alpha,1.0
,fit_intercept,True
,precompute,False
,copy_X,True
,max_iter,1000
,tol,0.0001
,warm_start,False
,positive,False
,random_state,None
,selection,'cyclic'


In [699]:
from sklearn.metrics import mean_squared_error
y_pred_lasso = lasso_model.predict(X_test)

rmse_lasso = np.sqrt(mean_squared_error(y_test, y_pred_lasso))
r2_lasso = lasso_model.score(X_test, y_test)
print(f"TEST - [Lasso Regr.] RMSE: {rmse_lasso:.4f} | R2: {r2_lasso:.4f}")
# We can see that from -48 percent score to 70 percent score our model is much bette now after using L1 regularization.

TEST - [Lasso Regr.] RMSE: 397202.4945 | R2: 0.4828


In [700]:
from sklearn.metrics import mean_squared_error
y_pred_lasso = lasso_model.predict(X_train)

rmse_lasso = np.sqrt(mean_squared_error(y_train, y_pred_lasso))
r2_lasso = lasso_model.score(X_train, y_train)
print(f"TRAIN - [Lasso Regr.] RMSE: {rmse_lasso:.4f} | R2: {r2_lasso:.4f}")

TRAIN - [Lasso Regr.] RMSE: 406515.4925 | R2: 0.4928


## Ridge

In [701]:
# Now we will use the L2 regularization tehnique
from sklearn.linear_model import Ridge
ridge_model = Ridge(alpha=50, max_iter=100, tol=0.1)
ridge_model.fit(X_train, y_train)

,alpha,50
,fit_intercept,True
,copy_X,True
,max_iter,100
,tol,0.1
,solver,'auto'
,positive,False
,random_state,None


In [702]:
from sklearn.metrics import mean_squared_error
y_pred_ridge = ridge_model.predict(X_test)

rmse_ridge = np.sqrt(mean_squared_error(y_test, y_pred_ridge))
r2_ridge = ridge_model.score(X_test,y_test)
print(f"TEST - [Ridge Regr.] RMSE: {rmse_ridge:.4f} | R2: {r2_ridge:.4f}")

# after using L2 regularization our model is also much better but it seems that L1 regularization is slightly better then L2 in this case.

TEST - [Ridge Regr.] RMSE: 396781.2319 | R2: 0.4839


In [703]:
from sklearn.metrics import mean_squared_error
y_pred_ridge = ridge_model.predict(X_train)

rmse_ridge = np.sqrt(mean_squared_error(y_train, y_pred_ridge))
r2_ridge = ridge_model.score(X_train,y_train)
print(f"TRAIN - [Ridge Regr.] RMSE: {rmse_ridge:.4f} | R2: {r2_ridge:.4f}")

TRAIN - [Ridge Regr.] RMSE: 410806.1078 | R2: 0.4820
